Capstone Project for Data Science

In [96]:
import pandas as pd
import numpy as np
print('Hello Capstone Project Course')

Hello Capstone Project Course


In [2]:
!conda install -c conda-forge geopy --yes

Solving environment: ...working... done

# All requested packages already installed.



In [6]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... done

## Package Plan ##

  environment location: D:\Anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1g             |       he774522_0         5.7 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         6.4 MB

The following NEW packages will be INSTALLED:

  altair             conda-forge/noarch::alt

# Week 3, Segmenting and Clustering Neighborhoods in Toronto

In [97]:
import json

from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

In [98]:
data_page = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(data_page)
df_toronto = df[0]
df_toronto

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [99]:
df_toronto = df_toronto[df_toronto['Borough'] != 'Not assigned']
df_toronto

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


## Task 1: Dataframe Preparation

In [100]:
Poslist = ['M5G', 'M2H','M4B','M1J','M4G','M4M','M1R','M9V','M9L','M5V','M1B','M5A']
df_pos = pd.DataFrame(columns = df_toronto.columns)
for row in df_toronto.index:
    if df_toronto['Postal Code'][row] in Poslist:
        df_pos = df_pos.append({'Postal Code' :df_toronto['Postal Code'][row],
                                               'Borough': df_toronto['Borough'][row],
                                             'Neighbourhood': df_toronto['Neighbourhood'][row] }, ignore_index = True)
df_pos


,Postal Code,Borough,Neighbourhood
0,M5A,Downtown Toronto,"Regent Park, Harbourfront"
1,M1B,Scarborough,"Malvern, Rouge"
2,M4B,East York,"Parkview Hill, Woodbine Gardens"
3,M4G,East York,Leaside
4,M5G,Downtown Toronto,Central Bay Street
5,M2H,North York,Hillcrest Village
6,M1J,Scarborough,Scarborough Village
7,M9L,North York,Humber Summit
8,M4M,East Toronto,Studio District
9,M1R,Scarborough,"Wexford, Maryvale"


In [101]:
df_pos.shape

(12, 3)

## Task 2 Acquire Coordinates for Postal Code

In [102]:
!conda install -c conda-forge geocoder --yes

Solving environment: ...working... done

# All requested packages already installed.



In [103]:
import geocoder

In [104]:
pos_coords = pd.read_csv('https://cocl.us/Geospatial_data')
pos_coords

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [105]:
df_pos = df_pos.join(pos_coords.set_index('Postal Code'), on = 'Postal Code')
df_pos

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
2,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
3,M4G,East York,Leaside,43.709060,-79.363452
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
5,M2H,North York,Hillcrest Village,43.803762,-79.363452
6,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
7,M9L,North York,Humber Summit,43.756303,-79.565963
8,M4M,East Toronto,Studio District,43.659526,-79.340923
9,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849


## Task 3 Segmenting Clustering

In [106]:
df_pos.dtypes

Postal Code       object
Borough           object
Neighbourhood     object
Latitude         float64
Longitude        float64
dtype: object

In [107]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent = 'toronto_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location. longitude
location

Location(Toronto, Golden Horseshoe, Ontario, Canada, (43.6534817, -79.3839347, 0.0))

In [108]:
map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 10)

for lat, lng, borough, neighbourhood in zip(df_pos['Latitude'], df_pos['Longitude'], df_pos['Borough'], df_pos['Neighbourhood']):
    label = '{},{}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat,lng],
        radius = 5,
        popup=label,
        color = 'blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

In [109]:
dt_data = df_pos[df_pos['Borough']=='Downtown Toronto'].reset_index(drop = True)
dt_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
2,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420


In [110]:
address = 'Downtown Toronto, ON'

geolocator = Nominatim(user_agent = 'toronto_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location. longitude
location

Location(Downtown Yonge, Old Toronto, Toronto, Golden Horseshoe, Ontario, M5B 2C3, Canada, (43.6563221, -79.3809161, 0.0))

In [111]:
map_dttoronto = folium.Map(location = [latitude, longitude], zoom_start = 11)

for lat, lng, label in zip(dt_data['Latitude'],dt_data['Longitude'],dt_data['Neighbourhood']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat,lng],
            radius=5,
            popup=label,
            color = 'blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity = 0.7,
            parse_html=False).add_to(map_dttoronto) 

map_dttoronto

In [112]:
CLIENT_ID = 'WBWCB0Y5FNG13MNUIZLERYJI1WLBPTGVPIZZCYVIZHRVPLDL'
CLIENT_SECRET = 'UMUGWMFE2DYYIJUYGEQYT23WIY04HGOAJ5IORUDXY4INE3B3'
VERSION = '20180605'
LIMIT = 100

In [113]:
dt_data.loc[0,'Neighbourhood']

'Regent Park, Harbourfront'

In [114]:
neigh_lat = dt_data.loc[0, 'Latitude']
neigh_lnt = dt_data.loc[0, 'Longitude']

neigh_name = dt_data.loc[0,'Neighbourhood']

In [115]:
limit = 100
radius = 300
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neigh_lat, 
    neigh_lnt, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=WBWCB0Y5FNG13MNUIZLERYJI1WLBPTGVPIZZCYVIZHRVPLDL&client_secret=UMUGWMFE2DYYIJUYGEQYT23WIY04HGOAJ5IORUDXY4INE3B3&v=20180605&ll=43.6542599,-79.3606359&radius=300&limit=100'

In [116]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f9873373f31f819379fd3e7'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 11,
  'suggestedBounds': {'ne': {'lat': 43.6569599027, 'lng': -79.35691110008916},
   'sw': {'lat': 43.6515598973, 'lng': -79.36436069991085}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',


In [117]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [118]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-118-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149


In [119]:
nearby_venues.shape[0]

11

In [120]:
def getNearbyVenues(names, latitudes, longitudes, radius=300):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [121]:
# type your answer here
dt_venues = getNearbyVenues(names=dt_data['Neighbourhood'], latitudes=dt_data['Latitude'],longitudes=dt_data['Longitude'])

Regent Park, Harbourfront
Central Bay Street
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


In [122]:
dt_venues.shape

(49, 7)

In [123]:
dt_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",5,5,5,5,5,5
Central Bay Street,33,33,33,33,33,33
"Regent Park, Harbourfront",11,11,11,11,11,11


In [124]:
len(dt_venues['Venue Category'].unique())

30

In [135]:
dt_onehot = pd.get_dummies(dt_venues[['Venue Category']], prefix = "",prefix_sep="")

dt_onehot['Neighborhood'] = dt_venues['Neighborhood']

fixed_columns = [dt_onehot.columns[-1]] + list(dt_onehot.columns[:-1])
dt_onehot = dt_onehot[fixed_columns]
dt_onehot.head()

,Neighborhood,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,Bakery,Bank,Bookstore,Breakfast Spot,Bubble Tea Shop,...,Japanese Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Park,Pharmacy,Pizza Place,Restaurant,Sandwich Place,Smoothie Shop,Spa
0,"Regent Park, Harbourfront",0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [126]:
dt_onehot.shape

(49, 31)

In [127]:
dt_grouped = dt_onehot.groupby('Neighborhood').mean().reset_index()
dt_grouped

,Neighborhood,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,Bakery,Bank,Bookstore,Breakfast Spot,Bubble Tea Shop,...,Japanese Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Park,Pharmacy,Pizza Place,Restaurant,Sandwich Place,Smoothie Shop,Spa
0,"CN Tower, King and Spadina, Railway Lands, Har...",0.2,0.2,0.2,0.2,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Central Bay Street,0.0,0.0,0.0,0.0,0.000000,0.030303,0.030303,0.000000,0.030303,...,0.030303,0.030303,0.030303,0.000000,0.030303,0.030303,0.060606,0.060606,0.030303,0.030303
2,"Regent Park, Harbourfront",0.0,0.0,0.0,0.0,0.090909,0.000000,0.000000,0.090909,0.000000,...,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909


In [128]:
dt_grouped.shape

(3, 31)

In [129]:
num_top_venues = 3
for hood in dt_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = dt_grouped[dt_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0  Airport Food Court   0.2
1      Airport Lounge   0.2
2    Airport Terminal   0.2


----Central Bay Street----
            venue  freq
0     Coffee Shop  0.36
1            Café  0.09
2  Sandwich Place  0.06


----Regent Park, Harbourfront----
        venue  freq
0  Food Truck  0.09
1    Bus Stop  0.09
2        Park  0.09




In [130]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [131]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = dt_grouped['Neighborhood']

for ind in np.arange(dt_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dt_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,Coffee Shop
1,Central Bay Street,Coffee Shop,Café,Sandwich Place,Restaurant,Italian Restaurant
2,"Regent Park, Harbourfront",Spa,History Museum,Bakery,Breakfast Spot,Bus Stop


In [132]:
# set number of clusters
kclusters = 3

dt_grouped_clustering = dt_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dt_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 2])

In [133]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

dt_data = dt_data.rename({'Neighbourhood':'Neighborhood'},axis = 1)

dt_merged = dt_data

dt_merged = dt_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

dt_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,Spa,History Museum,Bakery,Breakfast Spot,Bus Stop
1,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,Coffee Shop,Café,Sandwich Place,Restaurant,Italian Restaurant
2,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,Coffee Shop


In [134]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dt_merged['Latitude'], dt_merged['Longitude'],dt_merged['Neighborhood'], dt_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

KeyError: 'Cluster Labels'